###### 依赖模块安装

In [1]:
! pip install --upgrade --quiet langchain
! pip install --upgrade --quiet dashscope

# pdf文档的读取和ocr识别
! pip install --upgrade --quiet pypdf rapidocr-onnxruntime

# 向量检索引擎
! pip install --upgrade --quiet faiss-cpu

#####  DASHSCOPE 配置

In [2]:
from getpass import getpass
import os

# 如果没配置api key，获取api key
if os.getenv("DASHSCOPE_API_KEY") is None:
    os.environ["DASHSCOPE_API_KEY"] = getpass()

##### 引入相关包

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import DashScopeEmbeddings

from langchain_community.llms import Tongyi
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os

##### 数据准备

In [4]:
pdf_loader = PyPDFLoader('../1706.03762v7.pdf', extract_images=True)

text_splitter = RecursiveCharacterTextSplitter(
   chunk_size=1000, chunk_overlap=200, add_start_index=True
)

pages = pdf_loader.load_and_split()

##### 词嵌入并保存到向量数据库

In [5]:
embeddings = DashScopeEmbeddings(
   model="text-embedding-v1"
)

faiss_index = FAISS.from_documents(pages, embeddings)

# 保存到向量数据库中
faiss_index.save_local('documents.faiss')

##### 构建召回器和语言模型

In [6]:
faiss_index = FAISS.load_local('documents.faiss', embeddings, allow_dangerous_deserialization=True)

retriever = faiss_index.as_retriever(search_kwargs={"k": 6})

llm = Tongyi()

##### 构建问答模板

In [7]:
template = """利用以下上下文回答最后的问题。如果不知道答案，就说不知道，不要试图编造答案。

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
   | custom_rag_prompt
   | llm
   | StrOutputParser()
)

In [8]:
rag_chain.invoke("有没有关于attention的论文推荐？")

'可以参考这篇论文：“Attention Is All You Need”，作者包括Ashish Vaswani、Noam Shazeer、Niki Parmar等人。这篇论文提出了Transformer模型，这是一种完全基于注意力机制的新网络架构，摒弃了传统的循环和卷积层。实验结果显示，这种模型在机器翻译任务上表现优越，同时训练速度更快。您可以在以下链接找到更多详细信息：\n\n- [cs.CL] 2 Aug 2023 (arXiv:1706.03762v7)\n\n该论文还详细介绍了Transformer模型的设计理念和技术细节，包括缩放点积注意力（Scaled Dot-Product Attention）和多头注意力（Multi-Head Attention）等关键组件。'